In [35]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Myth of programming

Consider the following code snippets, which side is faster?

In [32]:
compare([do_render_code("arithmetics_3.c", show=["//START_MUL","//END_MUL"]),do_render_code("arithmetics_3.c", show=["//START_SHIFT","//END_SHIFT"])])

In [36]:
! make clean; make arithmetics_3
! echo "version,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses,branches,branch_miss_pred" > arithmetics_3.csv
for x in range(5):
    ! ./arithmetics_3 2097152 2>> arithmetics_3.csv
display_df_mono(render_csv("arithmetics_3.csv"))

rm -f arithmetics linked_list flip_array swap linked_list.s
make: 'arithmetics_3' is up to date.
2.728659 seconds
2.757061 seconds
259508292
2.720607 seconds
2.748658 seconds
259508292
2.739611 seconds
2.744067 seconds
259508292
2.710789 seconds
2.766690 seconds
259508292
2.709632 seconds
2.746297 seconds
259508292


,index,version,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses,branches,branch_miss_pred
0,0,Mul,21026214197,13866123859,0.659468,0.196447,2.723957,0.000021,211189,10009927902,1004460129,14874
1,1,Shift,23026636364,14073643464,0.611190,0.195900,2.757023,0.000018,213484,12010051118,1004542869,11026
2,2,Mul,21026990100,13867027011,0.659487,0.196178,2.720401,0.000022,224789,10010208329,1004594100,15034
3,3,Shift,23027676197,14075880760,0.611259,0.195272,2.748619,0.000020,238657,12010421991,1004722820,16196
4,4,Mul,20970823962,13897101815,0.662687,0.197129,2.739526,0.000015,153417,9983945218,1001223358,14431
5,5,Shift,23026340329,14061059437,0.610651,0.195150,2.744019,0.000018,221700,12009822352,1004526873,13332
6,6,Mul,21025574195,13889044188,0.660579,0.195169,2.710717,0.000022,224134,10009607863,1004375300,16165
7,7,Shift,23026216354,14078000459,0.611390,0.196523,2.766649,0.000019,227807,12009941316,1004455825,14835
8,8,Mul,21025521907,13884701977,0.660374,0.195146,2.709550,0.000021,205946,10009588189,1004361204,14276
9,9,Shift,23027398019,14070928884,0.611052,0.195173,2.746262,0.000019,224592,12010314523,1004676172,12716


In [2]:
compare([do_render_code("swap.c", show="regswap"),do_render_code("swap.c", show="xorswap")])

In [2]:
render_code("flip_array.c")

// flip_array.c:1-60 (60 lines)
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include "perfstats.h"

void (*swap)(int* a, int* b);
void regswap(int* a, int* b)
{
    int temp = *a;
    *a = *b;
    *b = temp;
}

void xorswap(int* a, int* b)
{
    *a ^= *b;
    *b ^= *a;
    *a ^= *b;
}

void flip_array(int64_t *a, int64_t *b, int64_t count)
{
    for(size_t i = 0; i < count; i++) {
    int64_t temp = a[i];
    a[i] = b[i];
    b[i] = temp;
    }
    return;
}

int main(int argc, char **argv)
{
    unsigned array_size = 131072;
    int *data;
    struct timeval time_start, time_end;
    // command: swap array_size method
    if(argc > 1)
        array_size = atoi(argv[1]);
    if(argc > 2 && atoi(argv[2])==1)
        swap = xorswap;
    else
        swap = regswap;
    data = (int *)malloc(sizeof(int)*array_size);
    for (unsigned i = 0; i < array_size; ++i)
        data[i] = rand();
   gettimeofday(&time_start, NULL);
    perfstats_init();
    perfstats_enable(1);
   for (unsigned i = 0; i < array_size; i++)
   {
        swap(&data[i],&data[array_size-1-i]);
   }
    perfstats_disable(1);
    perfstats_print();
    perfstats_deinit();
   gettimeofday(&time_end, NULL);
   fprintf(stderr, "data[array_size/2] = %d\n", data[rand()%array_size]);
   fprintf(stderr, "sorted %lf seconds\n",((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0);
   return 0;
}

In [26]:
! make clean; make flip_array
! echo "version,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses,branches,branch_miss_pred" > swap.csv
! echo "Version A"
for x in range(5):
    ! echo -n "Version A," >> swap.csv; ./flip_array 2097152 0 2>> swap.csv
! echo "Version B"
for x in range(5):
    ! echo -n "Version B," >> swap.csv; ./flip_array 2097152 1 2>> swap.csv
display_df_mono(render_csv("swap.csv"))

rm -f arithmetics linked_list flip_array swap linked_list.s
cc -O3 -DHAVE_LINUX_PERF_EVENT_H perfstats.c flip_array.c -o flip_array
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
cc -S -O3 -DHAVE_LINUX_PERF_EVENT_H  flip_array.c -o flip_array.s
Version A
data[array_size/2] = 903337577
flipped in 0.013741 seconds
data[array_size/2] = 903337577
flipped in 0.002362 seconds
data[array_size/2] = 903337577
flipped in 0.002244 seconds
data[array_size/2] = 903337577
flipped in 0.002585 seconds
data[array_size/2] = 903337577
flipped in 0.002325 seconds
Version B
data[array_size/2] = 903337577
flipped in 0.002733 seconds
data[array_size/2] = 903337577
flipped in 0

,index,version,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses,branches,branch_miss_pred
0,0,Version A,29406177,11149117,0.379142,0.315989,0.003523,0.015551,228562,14697664,6299438,292
1,1,Version A,29391627,11505590,0.391458,0.198165,0.002280,0.017683,259791,14691958,6297019,93
2,2,Version A,29382705,10930923,0.372019,0.198977,0.002175,0.017725,260367,14689211,6295316,107
3,3,Version A,29396962,12795545,0.435268,0.195459,0.002501,0.001836,26972,14694162,6297881,156
4,4,Version A,29381684,11351858,0.386358,0.198382,0.002252,0.017649,259246,14688875,6295136,113
5,5,Version B,33587305,13417566,0.399483,0.197651,0.002652,0.012442,261089,20984421,6297098,140
6,6,Version B,33578885,13210839,0.393427,0.196884,0.002601,0.012452,261263,20981272,6295683,115
7,7,Version B,33582060,13173920,0.392290,0.196297,0.002586,0.012467,261590,20982310,6296253,219
8,8,Version B,33579290,13123933,0.390834,0.200321,0.002629,0.012455,261331,20981709,6295699,268
9,9,Version B,33591179,13428442,0.399761,0.195332,0.002623,0.012460,261473,20985539,6297872,132


In [27]:
render_code("flip_array.s")

// flip_array.s:1-290 (290 lines)
	.file	"flip_array.c"
	.text
	.p2align 4
	.globl	regswap
	.type	regswap, @function
regswap:
.LFB39:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	movl	(%rsi), %edx
	movl	%edx, (%rdi)
	movl	%eax, (%rsi)
	ret
	.cfi_endproc
.LFE39:
	.size	regswap, .-regswap
	.p2align 4
	.globl	xorswap
	.type	xorswap, @function
xorswap:
.LFB40:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	xorl	(%rsi), %eax
	movl	%eax, (%rdi)
	xorl	(%rsi), %eax
	movl	%eax, (%rsi)
	xorl	%eax, (%rdi)
	ret
	.cfi_endproc
.LFE40:
	.size	xorswap, .-xorswap
	.p2align 4
	.globl	flip_array
	.type	flip_array, @function
flip_array:
.LFB41:
	.cfi_startproc
	endbr64
	movq	%rsi, %rcx
	testq	%rdx, %rdx
	je	.L4
	leaq	-1(%rdx), %rax
	cmpq	$1, %rax
	jbe	.L12
	leaq	15(%rdi), %rax
	subq	%rsi, %rax
	cmpq	$30, %rax
	ja	.L7
.L12:
	xorl	%eax, %eax
	.p2align 4,,10
	.p2align 3
.L8:
	movq	(%rdi,%rax,8), %rsi
	movq	(%rcx,%rax,8), %r8
	movq	%r8, (%rdi,%rax,8)
	movq	%rsi, (%rcx,%rax,8)
	addq	$1, %rax
	cmpq	%rax, %rdx
	jne	.L8
.L4:
	ret
	.p2align 4,,10
	.p2align 3
.L7:
	movq	%rdx, %rsi
	xorl	%eax, %eax
	shrq	%rsi
	salq	$4, %rsi
	.p2align 4,,10
	.p2align 3
.L9:
	movdqu	(%rdi,%rax), %xmm0
	movdqu	(%rcx,%rax), %xmm1
	movups	%xmm1, (%rdi,%rax)
	movups	%xmm0, (%rcx,%rax)
	addq	$16, %rax
	cmpq	%rax, %rsi
	jne	.L9
	movq	%rdx, %rax
	andq	$-2, %rax
	andl	$1, %edx
	je	.L4
	salq	$3, %rax
	addq	%rax, %rdi
	addq	%rax, %rcx
	movq	(%rdi), %rdx
	movq	(%rcx), %rax
	movq	%rax, (%rdi)
	movq	%rdx, (%rcx)
	ret
	.cfi_endproc
.LFE41:
	.size	flip_array, .-flip_array
	.section	.rodata.str1.1,"aMS",@progbits,1
.LC0:
	.string	"data[array_size/2] = %d\n"
.LC2:
	.string	"flipped in %lf seconds\n"
	.section	.text.startup,"ax",@progbits
	.p2align 4
	.globl	main
	.type	main, @function
main:
.LFB42:
	.cfi_startproc
	endbr64
	pushq	%r14
	.cfi_def_cfa_offset 16
	.cfi_offset 14, -16
	pushq	%r13
	.cfi_def_cfa_offset 24
	.cfi_offset 13, -24
	pushq	%r12
	.cfi_def_cfa_offset 32
	.cfi_offset 12, -32
	pushq	%rbp
	.cfi_def_cfa_offset 40
	.cfi_offset 6, -40
	pushq	%rbx
	.cfi_def_cfa_offset 48
	.cfi_offset 3, -48
	subq	$48, %rsp
	.cfi_def_cfa_offset 96
	movq	%fs:40, %rax
	movq	%rax, 40(%rsp)
	xorl	%eax, %eax
	cmpl	$1, %edi
	jg	.L19
	leaq	regswap(%rip), %rax
	movl	$524288, %edi
	movl	$131072, %r13d
	movq	%rax, swap(%rip)
	call	malloc@PLT
	movq	%rax, %r12
.L20:
	xorl	%ebx, %ebx
	.p2align 4,,10
	.p2align 3
.L25:
	call	rand@PLT
	movl	%eax, (%r12,%rbx,4)
	addq	$1, %rbx
	cmpl	%r13d, %ebx
	jb	.L25
	movq	%rsp, %rdi
	xorl	%esi, %esi
	leal	-1(%r13), %ebx
	movq	%r12, %rbp
	call	gettimeofday@PLT
	call	perfstats_init@PLT
	movl	$1, %edi
	call	perfstats_enable@PLT
	.p2align 4,,10
	.p2align 3
.L28:
	movl	%ebx, %eax
	subl	$1, %ebx
	movq	%rbp, %rdi
	addq	$4, %rbp
	leaq	(%r12,%rax,4), %rsi
	call	*swap(%rip)
	cmpl	$-1, %ebx
	jne	.L28
.L27:
	movl	$1, %edi
	call	perfstats_disable@PLT
	xorl	%eax, %eax
	call	perfstats_print@PLT
	call	perfstats_deinit@PLT
	xorl	%esi, %esi
	leaq	16(%rsp), %rdi
	call	gettimeofday@PLT
	movq	stderr(%rip), %rsi
	movl	$10, %edi
	call	fputc@PLT
	call	rand@PLT
	xorl	%edx, %edx
	movl	$2, %edi
	leaq	.LC0(%rip), %rsi
	divl	%r13d
	xorl	%eax, %eax
	movl	(%r12,%rdx,4), %edx
	call	__printf_chk@PLT
	pxor	%xmm0, %xmm0
	movl	$2, %edi
	leaq	.LC2(%rip), %rsi
	imulq	$1000000, 16(%rsp), %rax
	addq	24(%rsp), %rax
	imulq	$1000000, (%rsp), %rdx
	addq	8(%rsp), %rdx
	subq	%rdx, %rax
	cvtsi2sdq	%rax, %xmm0
	movl	$1, %eax
	divsd	.LC1(%rip), %xmm0
	call	__printf_chk@PLT
	movq	40(%rsp), %rax
	subq	%fs:40, %rax
	jne	.L34
	addq	$48, %rsp
	.cfi_remember_state
	.cfi_def_cfa_offset 48
	xorl	%eax, %eax
	popq	%rbx
	.cfi_def_cfa_offset 40
	popq	%rbp
	.cfi_def_cfa_offset 32
	popq	%r12
	.cfi_def_cfa_offset 24
	popq	%r13
	.cfi_def_cfa_offset 16
	popq	%r14
	.cfi_def_cfa_offset 8
	ret
.L19:
	.cfi_restore_state
	movl	%edi, %ebx
	movq	8(%rsi), %rdi
	movq	%rsi, %rbp
	movl	$10, %edx
	xorl	%esi, %esi
	call	strtol@PLT
	movq	%rax, %r14
	movl	%eax, %r13d
	cmpl	$2, %ebx
	je	.L23
	movq	16(%rbp), %rdi
	xorl	%esi, %esi
	movl	$10, %edx
	call	strtol@PLT
	

In [29]:
compare([do_render_code("arithmetics.c", show="init_double"),do_render_code("arithmetics.c", show="init_int"),do_render_code("arithmetics.c", show="init_int_mul"),do_render_code("arithmetics.c", show="init_int_memory")])

In [30]:
! make arithmetics
! ./arithmetics 68157440 0
! ./arithmetics 68157440 1
! ./arithmetics 68157440 2
! ./arithmetics 68157440 3

cc -O4 -DHAVE_LINUX_PERF_EVENT_H arithmetics.c -o arithmetics
arithmetics.c: In function ‘main’:
arithmetics.c:72:37: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘int64_t’ {aka ‘long int’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
   72 |    fprintf(stderr, "data[rand()] = %d\n", target[rand()%array_size]);
      |                                    ~^     ~~~~~~~~~~~~~~~~~~~~~~~~~
      |                                     |           |
      |                                     int         int64_t {aka long int}
      |                                    %ld
cc -O4 -DHAVE_LINUX_PERF_EVENT_H -S arithmetics.c 
arithmetics.c: In function ‘main’:
arithmetics.c:72:37: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘int64_t’ {aka ‘long int’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
   72 |    fprintf(stderr, "data[rand()] 

In [39]:
!make -C ../intro
print("With sorting")
! time ../intro/arraySort 262144 1000 1
print("Without sorting")
! time ../intro/arraySort 262144 1000 0

make: Entering directory '/nfshome/htseng/courses/CS203/demo/intro'
gcc calculate_sum.c -c 
g++ -O0 arraySort.cpp calculate_sum.o -o arraySort
make: *** No rule to make target 'quicksort_cpu.cpp', needed by 'quicksort_cpu'.  Stop.
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/intro'
With sorting
sum = 127161197

real	0m0.179s
user	0m0.174s
sys	0m0.004s
Without sorting
sum = 127161197

real	0m0.591s
user	0m0.590s
sys	0m0.001s


## Linked List v.s. Array

In [8]:
compare([do_render_code("linked_list.c", show="sum_of_array"),do_render_code("linked_list.c", show="depth_of_list")])

In [10]:
! make clean; make
! echo "size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "4096,1," >> stats.csv
! ./linked_list 4096 1
! echo -n "4096,0," >> stats.csv
! ./linked_list 4096 0

rm -f arithmetics linked_list flip_array swap linked_list.s
cc -O4 -DHAVE_LINUX_PERF_EVENT_H arithmetics.c -o arithmetics
arithmetics.c: In function ‘main’:
arithmetics.c:72:37: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘int64_t’ {aka ‘long int’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
   72 |    fprintf(stderr, "data[rand()] = %d\n", target[rand()%array_size]);
      |                                    ~^     ~~~~~~~~~~~~~~~~~~~~~~~~~
      |                                     |           |
      |                                     int         int64_t {aka long int}
      |                                    %ld
cc -O4 -DHAVE_LINUX_PERF_EVENT_H -S arithmetics.c 
arithmetics.c: In function ‘main’:
arithmetics.c:72:37: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘int64_t’ {aka ‘long int’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat

In [16]:
display_df_mono(render_csv("stats.csv"))

,size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,4096,1,0,0,nan,inf,0.000090,nan,0,0,0,0
1,4096,0,0,0,nan,inf,0.000003,nan,0,0,0,0


In [9]:
render_code("linked_list.s", show=["sum_of_array:",".LFE40:"])

// linked_list.s:33-139 (107 lines)
sum_of_array:
.LFB40:
	.cfi_startproc
	endbr64
	testl	%esi, %esi
	jle	.L17
	leal	-1(%rsi), %ecx
	cmpl	$3, %ecx
	jbe	.L18
	movl	%ecx, %edx
	pxor	%xmm2, %xmm2
	pxor	%xmm4, %xmm4
	movq	%rdi, %rax
	shrl	$2, %edx
	movdqa	%xmm2, %xmm5
	salq	$6, %rdx
	addq	%rdi, %rdx
	.p2align 4,,10
	.p2align 3
.L10:
	movdqu	16(%rax), %xmm3
	movdqu	48(%rax), %xmm6
	movdqa	%xmm4, %xmm7
	addq	$64, %rax
	movdqu	-64(%rax), %xmm0
	movdqu	-32(%rax), %xmm1
	shufps	$136, %xmm3, %xmm0
	shufps	$136, %xmm6, %xmm1
	movdqu	-8(%rax), %xmm3
	shufps	$136, %xmm1, %xmm0
	movdqu	-24(%rax), %xmm1
	movdqu	-40(%rax), %xmm6
	pcmpgtd	%xmm0, %xmm7
	punpcklqdq	%xmm3, %xmm1
	pcmpeqd	%xmm5, %xmm1
	pcmpeqd	%xmm4, %xmm1
	pshufd	$177, %xmm1, %xmm3
	por	%xmm1, %xmm3
	movdqu	-56(%rax), %xmm1
	punpcklqdq	%xmm6, %xmm1
	pcmpeqd	%xmm5, %xmm1
	pcmpeqd	%xmm4, %xmm1
	pshufd	$177, %xmm1, %xmm6
	por	%xmm1, %xmm6
	movdqa	%xmm0, %xmm1
	punpckhdq	%xmm7, %xmm0
	punpckldq	%xmm7, %xmm1
	pand	%xmm3, %xmm0
	pand	%xmm6, %xmm1
	paddq	%xmm1, %xmm0
	paddq	%xmm0, %xmm2
	cmpq	%rdx, %rax
	jne	.L10
	movdqa	%xmm2, %xmm0
	andl	$-4, %ecx
	psrldq	$8, %xmm0
	movl	%ecx, %edx
	paddq	%xmm0, %xmm2
	movq	%xmm2, %rax
.L9:
	movslq	%edx, %rcx
	salq	$4, %rcx
	leaq	(%rdi,%rcx), %r8
	cmpq	$0, 8(%r8)
	je	.L11
	movslq	(%r8), %r8
	addq	%r8, %rax
.L11:
	leal	1(%rdx), %r8d
	cmpl	%r8d, %esi
	jle	.L7
	leaq	16(%rdi,%rcx), %r8
	cmpq	$0, 8(%r8)
	je	.L13
	movslq	(%r8), %r8
	addq	%r8, %rax
.L13:
	leal	2(%rdx), %r8d
	cmpl	%r8d, %esi
	jle	.L7
	leaq	32(%rdi,%rcx), %r8
	cmpq	$0, 8(%r8)
	je	.L14
	movslq	(%r8), %r8
	addq	%r8, %rax
.L14:
	addl	$3, %edx
	cmpl	%edx, %esi
	jle	.L7
	leaq	48(%rdi,%rcx), %rdx
	cmpq	$0, 8(%rdx)
	je	.L7
	movslq	(%rdx), %rdx
	addq	%rdx, %rax
	ret
	.p2align 4,,10
	.p2align 3
.L17:
	xorl	%eax, %eax
.L7:
	ret
.L18:
	xorl	%eax, %eax
	xorl	%edx, %edx
	jmp	.L9
	.cfi_endproc
.LFE40:

The main loop between L10 and L9 has two 